In [1]:
import keras
import numpy as np
from pond.tensor import NativeTensor, PrivateEncodedTensor, PublicEncodedTensor
from pond.nn import Dense, Sigmoid, ReluExact, Reveal, Diff, Softmax, CrossEntropy, Sequential, DataLoader, Conv2D, AveragePooling2D, Flatten
from keras.utils import to_categorical
# raise error if overflow or underflow occurs
# def err_handler(type, flag):
#     print("Floating point error (%s), with flag %s" % (type, flag))
# saved_handler = np.seterrcall(err_handler)
# np.seterr(all='call')
_ = np.seterr(over='raise')
_ = np.seterr(invalid='raise')

# init functions
truncated_normal = lambda shp: scipy.stats.truncnorm.rvs(-2,2,loc=0,scale=0.1,size=shp)
normal = lambda shp: np.random.normal(scale=0.1, size=shp)
# samples1 = truncated_normal((100,10))
# samples2 = normal((100,10))

/home/ruben/anaconda2/envs/py36/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## read data

In [2]:
# read data
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = x_train[:,np.newaxis,:,:] / 255.0
x_test = x_test[:,np.newaxis,:,:] / 255.0
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

## Convnet using NativeTensor

In [3]:
convnet_shallow = Sequential([
    Conv2D((4, 4, 1, 20), strides=2, padding=1, filter_init=lambda shp: np.random.normal(scale=0.1, size=shp)),
    ReluExact(),
    AveragePooling2D(pool_size=(2,2)),
    Flatten(),
    Dense(10, 980),
    Reveal(),
    Softmax()
])

convnet_deep = Sequential([
    Conv2D((4, 4, 1, 20), strides=2, padding=1, filter_init=lambda shp: np.random.normal(scale=0.1, size=shp)),
    ReluExact(),
    AveragePooling2D(pool_size=(2,2)),
    Conv2D((3, 3, 20, 20), strides=2, padding=1, filter_init=lambda shp: np.random.normal(scale=0.1, size=shp)),
    ReluExact(),
    AveragePooling2D(pool_size=(2,2)),
    Flatten(),
    Dense(10, 80),
    Reveal(),
    Softmax()
])


In [4]:
convnet_deep.initialize()
convnet_deep.fit(
    x_train=DataLoader(x_train, wrapper=NativeTensor),
    y_train=DataLoader(y_train, wrapper=NativeTensor),
    x_valid=DataLoader(x_test, wrapper=NativeTensor),
    y_valid=DataLoader(y_test, wrapper=NativeTensor),
    loss=CrossEntropy(),
    epochs=3,
    batch_size=32,
    verbose=0,
)

TypeError: 'OutStream' object is not callable

TypeError: 'tuple' object is not callable

## PublicEncodedTensor

In [ ]:
convnet_shallow.initialize()
convnet_shallow.fit(
    x_train=DataLoader(x_train, wrapper=PublicEncodedTensor),
    y_train=DataLoader(y_train, wrapper=PublicEncodedTensor),
    x_valid=DataLoader(x_test, wrapper=PublicEncodedTensor),
    y_valid=DataLoader(y_test, wrapper=PublicEncodedTensor),
    loss=CrossEntropy(),
    epochs=1,
    batch_size=32,
    verbose=1,
)

## PrivateEncodedTensor

In [ ]:
convnet_shallow.initialize()
convnet_shallow.fit(
    x_train=DataLoader(x_train, wrapper=PrivateEncodedTensor), 
    y_train=DataLoader(y_train, wrapper=PrivateEncodedTensor), 
    loss=CrossEntropy(), 
    epochs=1,
    batch_size=32,
    verbose=1
)